In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
loan = pd.read_csv("C:/Users/USER/Desktop/프로젝트/작은 도서관 운영방안 추천/loan.csv")

In [ ]:
loan.drop(['Unnamed: 0'], axis = 1, inplace=True)
loan

In [ ]:
unique_values = loan['userInfo'].unique()
print(unique_values)

In [ ]:
sex = input('성별을 입력하세요:')
city = input('지역을 입력하세요:')
age = input('나이를 입력하세요:')
category = input('카테고리를 입력하세요:')
# 'userInfo' 열에서  sex, city, age가 들어간 모든 행 가져오기
condition = loan['userInfo'].str.contains(sex) & loan['userInfo'].str.contains(city) & loan['userInfo'].str.contains(age) & loan['userInfo'].str.contains(category)
filtered_data = loan[condition]

title_list = filtered_data['서명'].tolist()
title_list

In [ ]:
for title in title_list:
    print(title)

In [ ]:
driver = webdriver.Chrome("C:/chromedriver.exe")   #chromedriver 위치

예스24 크롤링
어디 올릴 때 아이디랑 비밀번호 지우고 올리기!!!!!!!!!!!!!!!!

In [ ]:
# 웹사이트 열기
driver.get('https://www.yes24.com/Templates/FTLogin.aspx')

# 로그인 정보 입력
username = driver.find_element_by_id('SMemberID')
password = driver.find_element_by_id('SMemberPassword')

username.send_keys('')        #아이디 입력
password.send_keys('')  #비밀번호 입력

# 로그인 버튼 클릭
login_button = driver.find_element_by_css_selector('#btnLogin > span > em').click()

# 로그인 후 페이지가 로드될 때까지 잠시 대기
time.sleep(5)

loan_name = []
loan_category = []
loan_info = []

# 각 제목을 검색창에 입력하고 검색 실행
for title in title_list:
    # 검색창 요소 찾기
    search = driver.find_element_by_id('query')

    # 텍스트를 선택하여 덮어쓰기
    search.send_keys(Keys.CONTROL + "a")  # 텍스트 선택
    search.send_keys(Keys.DELETE)  # 선택된 텍스트 삭제
    search.send_keys(title)  # 새로운 제목 입력

    # 검색 버튼 클릭
    search_button = driver.find_element_by_css_selector('#yesSForm > fieldset > span.schBtn > button')
    search_button.click()

    try:
        # 첫 번째로 나온 책 클릭
        elem = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#yesSchList > li > div > div.item_info > div.info_row.info_name > a.gd_name"))
        )
        elem.click()
        # 기다리는 중인 다른 조건 처리 (예: 원하는 요소가 로드될 때까지 대기)
    except Exception as e:
        print(f"Exception: 찾으시는 책이 없습니다.")
        continue  # 오류 발생 시 다음 제목으로 넘어가기


     # 책 정보 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    loan_book_name_element = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2')
    loan_book_name = loan_book_name_element.text.strip() if loan_book_name_element else 'No Name Found'
    loan_name.append(loan_book_name)

    # 책 카테고리(분야) 가져오기
    loan_book_category_element = soup.select_one('#infoset_goodsCate > div.infoSetCont_wrap > dl > dd')
    loan_book_category = loan_book_category_element.text.strip() if loan_book_category_element else 'No Area Found'
    if '>' or '/' or "\n" in loan_book_category:
        loan_book_category = loan_book_category.replace('>', '')
        loan_book_category = loan_book_category.replace('/', '')
        loan_book_category = loan_book_category.replace('\n', ' ')
        loan_category.append(loan_book_category)

    # 책 정보 가져오기
    loan_book_info_element = soup.select_one('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div')
    loan_book_info = loan_book_info_element.text.strip() if loan_book_info_element else 'No Info Found'
    if "<b>" or "</b>" or "<br/>" or "\n" in book_info:
        loan_book_info = loan_book_info.replace('<b>', ' ')
        loan_book_info = loan_book_info.replace('</b>', ' ')
        loan_book_info = loan_book_info.replace('<br/>', ' ')
        loan_book_info = loan_book_info.replace('\n', ' ')
    loan_info.append(loan_book_info)

In [ ]:
loan = pd.DataFrame({'name' : loan_name, 'category' : loan_category, 'info' : loan_info})
loan

In [ ]:
sale_names = []
sale_categories = []
sale_infos = []

for page_number in range(1, 6):
    url = 'https://www.yes24.com/24/Category/NewProductList/001?SumGb=04&PageNumber=' + str(page_number)
    driver.get(url)

    for row_number in range(1, 80, 2):
        try:
            driver.get(url)
            driver.implicitly_wait(time_to_wait=10)
            book_element = driver.find_element_by_css_selector("#category_layout > tbody > tr:nth-child(" + str(row_number) + ") > td.goodsTxtInfo > p:nth-child(1) > a:nth-child(1)").click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2')))

            # 책 정보 가져오기
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            sale_book_name_element = soup.select_one('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2')
            sale_book_name = sale_book_name_element.text.strip() if sale_book_name_element else 'No Name Found'
            sale_names.append(sale_book_name)

            # 책 카테고리(분야) 가져오기
            sale_book_category_element = soup.select_one('#infoset_goodsCate > div.infoSetCont_wrap > dl > dd')
            sale_book_category = sale_book_category_element.text.strip() if sale_book_category_element else 'No Area Found'
            if '>' in sale_book_category or '/' in sale_book_category or "\n" in sale_book_category:
                sale_book_category = sale_book_category.replace('>', '').replace('/', '').replace('\n', ' ')
            sale_categories.append(sale_book_category)

            # 책 정보 가져오기
            sale_book_info_element = soup.select_one('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt > div')
            sale_book_info = sale_book_info_element.text.strip() if sale_book_info_element else 'No Info Found'
            if "<b>" in sale_book_info or "</b>" in sale_book_info or "<br/>" in sale_book_info or "\n" in sale_book_info:
                sale_book_info = sale_book_info.replace('<b>', ' ').replace('</b>', ' ').replace('<br/>', ' ').replace('\n', ' ')
            sale_infos.append(sale_book_info)
        except Exception as e:
            print(f"Error: 찾으시는 책이 없습니다.")
            continue

In [ ]:
sale = pd.DataFrame({'name' : sale_names, 'category' : sale_categories, 'info' : sale_infos})
sale

In [ ]:
sale['name'].unique()

In [ ]:
sale['category'].unique()

In [ ]:
sale['info'].unique()

In [ ]:
# 두 데이터프레임 합치기
data = pd.concat([sale, loan], ignore_index=True)
data

In [ ]:
pip install konlpy

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from collections import Counter

okt = Okt()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
from tqdm.notebook import tqdm

temp_list = []
for i in tqdm(data.index):
    morph = okt.morphs(data.loc[i]['info'])
    temp_list = temp_list + morph

In [ ]:
len(temp_list)

In [ ]:
result = Counter(temp_list)
sorted_dict = sorted(result.items(), key=lambda item:item[1], reverse=True)
print(len(sorted_dict))
print(sorted_dict)

In [ ]:
stopword = []

with open("C:/Users/USER/Desktop/박수빈/stopword.txt", 'r', encoding='utf-8') as file:    #불용어 파일 위치
    for line in file:
        stopword.extend(line.strip().split())

print(stopword)

In [ ]:
stopword = stopword +  ['.', ',', '"', ':', '‘', '-','<','>', '에서', '이다', '을', '의', '를', '에','이','는','은','들', '한', '가', '에서', '그', '으로','로', '과','와','인','더','에게','수','도','다','게','있다', 'No', 'Info', 'Found']

In [ ]:
morphs_lst = []
for i in tqdm(data.index):
    morphs = ""
    morph = okt.morphs(data.loc[i]['info'])
    for txt in morph:
        if txt not in stopword:
            morphs = morphs + " " + txt
    morphs_lst.append(morphs)

data['morphs'] = morphs_lst
data.head()

In [ ]:
data.tail(20)

In [ ]:
data['name'].unique()

In [ ]:
tfidf = TfidfVectorizer()
#morphs열에 대해서 tf-idf 수정
tfidf_matrix = tfidf.fit_transform(data['morphs'])
print(tfidf_matrix.shape)

In [ ]:
#코사인 유사도를 사용하면 바로 위의 ()를 구한다
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

In [ ]:
cosine_sim.shape

In [ ]:
indices = pd.Series(data.index, index=data['name']).drop_duplicates()
print(indices.head())

In [ ]:
idx = indices['의존을 배우다']
print(idx)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(enumerate(cosine_sim[idx]), key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    book_indices = []
    for i in sim_scores:
        book_indices.append(i[0])
    return data['name'].iloc[book_indices]

In [ ]:
name_list = data['name'][-5:].tolist()
name_list

In [ ]:
book_name = []

for book in name_list:
    recommendations = get_recommendations(book)
    book_name.extend(recommendations)  # extend를 사용하여 리스트를 확장하여 book_name 리스트에 추가

In [ ]:
book_name = list(set(book_name))
print(book_name)